In [ ]:
class simplex:
  def solve(self, A_inv, x, k):
    n = len(A_inv)
    k -= 1 # для лучшей индексации
    # ШАГ 1
    l = np.dot(A_inv, x)
    li = l[k] # находим i-ую компоненту вектора l
    if (li == 0): # проверка компоненты на равенство нолю
      return "Матрица не может быть обратимой"
    # ШАГ 2
    l1 = np.copy(l) # получаем вектор l с волной
    l1[k] = -1 # заменяем i-ую компоненту вектора l с волной -1
    # ШАГ 3
    l2 = np.dot(-1/li, l1) # получаем вектор l с шапочкой
    # ШАГ 4
    E = np.eye(n) # создаем единичную матрицу
    Q = np.copy(E)
    #print(l2)
    Q[:,k] = l2.transpose() # создаем матрицу Q
    z = np.eye(n)
    for i in range(n):
      for j in range(n):
        z[i][j] = Q[i][i]*A_inv[i][j]
        if (i != k):
          z[i][j] += Q[i][k]*A_inv[k][j]
    return z

  def get_matrix(self, av, Jb):
    a = np.eye(len(Jb))
    j = 0
    for i in Jb:
      a[:, j] = av[:, i - 1]
      j += 1
    a = np.linalg.inv(a)
    return a
    
  def begin_simplex(self, A, b):
    n = len(A[0])
    m = len(b)
    for i in range(m):
      if b[i] < 0:
        b[i] *= -1
        A[i] *= -1
    E = np.eye(m)
    Av = np.empty([m, n + m])
    for i in range(m):
      Av[i] = np.append(A[i], E[i])
  
    x = np.append(np.zeros(n), b)
  
    Jb = np.array([n + i + 1 for i in range(m)])
    c = [0 if i < n else -1 for i in range(n + m)]
    x, jb = self.main_simplex(Av, c, x, Jb)
    print(x,jb)
    for i in x[n:]:
      if abs(i) > 10 ** -6:
        return 'Несовместна'
    while True:
      k = -1
      for i in range(len(jb)):
        if (jb[i] > n):
          k = i
      if k == -1:
        return x[:n], jb
        break
      print(k)
      jnb = []
      for i in range(1, n + 1):
        if not i in jb:
          jnb.append(i)
      print(jnb)
      avb_inv = self.get_matrix(Av, jb)
      flag = False
      for i in jnb:
        l = np.dot(avb_inv, Av[:,i - 1])
        print(i, l)
        if abs(l[k]) < 10 ** -6:
          jb[k] = i
          flag = True
          break
      print(jb)
      if not flag:
        c = c[:-1]
        jb = jb[:-1]
        A = A[:-1]
        Av = Av[:-1]
        b = b[:-1]

  def main_simplex(self, A, c, x, jb):
    iteration = 0
    while True:
      iteration += 1
      if iteration == 1:
        ab = np.array([np.copy(A[:,i-1]) for i in jb]).transpose()
        A_i = np.linalg.inv(ab)
      else:
        ab[:,position_min_tetta] = A[:, jb[position_min_tetta] - 1]
        A_i = self.solve(A_i, ab[:,position_min_tetta], position_min_tetta + 1)
      cb = np.array([c[i - 1] for i in jb])
      u = np.dot(cb, A_i)
      delta = np.dot(u, A) -c
      j0=0
      if min(delta) >= 0:
      
        return (x, jb)
      while delta[j0]>=0:
        j0 += 1
      z = np.dot(A_i, A[:,j0])
      tetta = [float(x[jb[i] - 1])/float(z[i]) if z[i]> 0 else np.inf for i in range(len(jb))]
      tetta0 = np.min(tetta)
      if tetta0 is np.inf:
        print("целевая функция неограничена на множестве допустимых планов")
      position_min_tetta = np.where(tetta == tetta0)[0][0]
      jb_new = np.copy(jb)
      jb_new[position_min_tetta] = j0 + 1
      x_new = np.copy(x)
      j = 0
      x_new = x_new.astype(np.float64)
      for i in jb:
        x_new[i - 1] = float(x[i - 1]) - tetta0*float(z[j])
        j+=1
      x_new[j0] = tetta0
      x = x_new
      jb = jb_new
        
  


In [ ]:

def solve(A_inv, x, k):
  n = len(A_inv)
  k -= 1 # для лучшей индексации
  # ШАГ 1
  l = np.dot(A_inv, x)
  li = l[k] # находим i-ую компоненту вектора l
  if (li == 0): # проверка компоненты на равенство нолю
    return "Матрица не может быть обратимой"
  # ШАГ 2
  l1 = np.copy(l) # получаем вектор l с волной
  l1[k] = -1 # заменяем i-ую компоненту вектора l с волной -1
  # ШАГ 3
  l2 = np.dot(-1/li, l1) # получаем вектор l с шапочкой
  # ШАГ 4
  E = np.eye(n) # создаем единичную матрицу
  Q = np.copy(E)
  #print(l2)
  Q[:,k] = l2.transpose() # создаем матрицу Q
  z = np.eye(n)
  for i in range(n):
    for j in range(n):
      z[i][j] = Q[i][i]*A_inv[i][j]
      if (i != k):
        z[i][j] += Q[i][k]*A_inv[k][j]
  return z

In [ ]:
from math import ceil
def get_f(a):
  return 1 - (ceil(a) - a)
  

In [ ]:
from scipy.optimize import linprog
import numpy as np
from math import trunc
def gomori(c, A, b, bounds):
  c *= -1
  while True:
    
    res = linprog( c, A, b, bounds = bound, method = 'simplex')
    
    x = res.x
    Jb = []
    for i in range(len(x)):
      if x[i] != 0:
        Jb.append(i + 1)
    
    while len(Jb) < len(A):
      for i in range(len(A[0])):
        if i + 1 not in Jb:
          Jb.append(i + 1)
          break
    k = None
    fl = 0
    for i in range(len(x)):
      
      if abs(round(x[i]) - x[i]) > 10 ** -6:
        k = i
        fl = 1
        break
    if fl == 0:
      return x
    
    ab = np.array([np.copy(A[:,i-1]) for i in Jb]).transpose()
    
    ab_inv = np.linalg.inv(ab)
    
    l = [0 for i in range(len(Jb))]
    l[k] = 1
    y = np.dot(l, ab_inv)
    small_a = []
    for i in range(len(A[0])):
      small_a.append(np.dot(y, A[:,i]))
    
    small_b = np.dot(y, b)
    Jnb = [i for i in range(1, len(c) + 1) if i not in Jb]
    new_a = [0 for _ in range(len(A[0]))]
    for i in range(len(Jnb)):
      new_a[Jnb[i] - 1] = -get_f(small_a[Jnb[i] - 1])
    new_a += [1]
    arr = A.tolist()
    for i in arr:
      i += [0]
    arr.append(new_a)
    A = np.array(arr)
    arr = c.tolist()
    arr += [0]
    c = np.array(arr)
    arr = b.tolist()
    arr += [-get_f(small_b)]
    b = np.array(arr)
    bound.append([0, None])
    


In [ ]:
c = np.array([21, 11,0])
A = np.array([[7, 4,1]])
b = np.array([13])
bound = [[0, None], [0, None],[0,None]]

print(gomori(c, A, b,bound))


[0. 3. 1. 1. 0.]


In [ ]:
"""
Read-me:
Call functions in this order:
    problem = gen_matrix(v,c)
    constrain(problem, string)
    obj(problem, string)
    maxz(problem)
gen_matrix() produces a matrix to be given constraints and an objective function to maximize or minimize.
    It takes var (variable number) and cons (constraint number) as parameters.
    gen_matrix(2,3) will create a 4x7 matrix by design.
constrain() constrains the problem. It takes the problem as the first argument and a string as the second. The string should be
    entered in the form of 1,2,G,10 meaning 1(x1) + 2(x2) >= 10.
    Use 'L' for <= instead of 'G'
Use obj() only after entering all constraints, in the form of 1,2,0 meaning 1(x1) +2(x2) +0
    The final term is always reserved for a constant and 0 cannot be omitted.
Use maxz() to solve a maximization LP problem. Use minz() to solve a minimization problem.
Disclosure -- pivot() function, subcomponent of maxz() and minz(), has a couple bugs. So far, these have only occurred when
    minz() has been called.
"""

import numpy as np

# generates an empty matrix with adequate size for variables and constraints.
def gen_matrix(var,cons):
    tab = np.zeros((cons+1, var+cons+2))
    return tab

# checks the furthest right column for negative values ABOVE the last row. If negative values exist, another pivot is required.
def next_round_r(table):
    m = min(table[:-1,-1])
    if m>= 0:
        return False
    else:
        return True

# checks that the bottom row, excluding the final column, for negative values. If negative values exist, another pivot is required.
def next_round(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m>=0:
        return False
    else:
        return True

# Similar to next_round_r function, but returns row index of negative element in furthest right column
def find_neg_r(table):
    # lc = number of columns, lr = number of rows
    lc = len(table[0,:])
    # search every row (excluding last row) in final column for min value
    m = min(table[:-1,lc-1])
    if m<=0:
        # n = row index of m location
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

#returns column index of negative element in bottom row
def find_neg(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        # n = row index for m
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

# locates pivot element in tableu to remove the negative element from the furthest right column.
def loc_piv_r(table):
        total = []
        # r = row index of negative entry
        r = find_neg_r(table)
        # finds all elements in row, r, excluding final column
        row = table[r,:-1]
        # finds minimum value in row (excluding the last column)
        m = min(row)
        # c = column index for minimum entry in row
        c = np.where(row == m)[0][0]
        # all elements in column
        col = table[:-1,c]
        # need to go through this column to find smallest positive ratio
        for i, b in zip(col,table[:-1,-1]):
            # i cannot equal 0 and b/i must be positive.
            if i**2>0 and b/i>0:
                total.append(b/i)
            else:
                # placeholder for elements that did not satisfy the above requirements. Otherwise, our index number would be faulty.
                total.append(0)
        element = max(total)
        for t in total:
            if t > 0 and t < element:
                element = t
            else:
                continue

        index = total.index(element)
        return [index,c]
# similar process, returns a specific array element to be pivoted on.
def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if i**2>0 and b/i>0:
                total.append(b/i)
            else:
                # placeholder for elements that did not satisfy the above requirements. Otherwise, our index number would be faulty.
                total.append(0)
        element = max(total)
        for t in total:
            if t > 0 and t < element:
                element = t
            else:
                continue

        index = total.index(element)
        return [index,n]

# Takes string input and returns a list of numbers to be arranged in tableu
def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

# The final row of the tablue in a minimum problem is the opposite of a maximization problem so elements are multiplied by (-1)
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]
    return table

# generates x1,x2,...xn for the varying number of variables.
def gen_var(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

# pivots the tableau such that negative elements are purged from the last row and last column
def pivot(row,col,table):
    # number of rows
    lr = len(table[:,0])
    # number of columns
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0: #new
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

# checks if there is room in the matrix to add another constraint
def add_cons(table):
    lr = len(table[:,0])
    # want to know IF at least 2 rows of all zero elements exist
    empty = []
    # iterate through each row
    for i in range(lr):
        total = 0
        for j in table[i,:]:
            # use squared value so (-x) and (+x) don't cancel each other out
            total += j**2
        if total == 0:
            # append zero to list ONLY if all elements in a row are zero
            empty.append(total)
    # There are at least 2 rows with all zero elements if the following is true
    if len(empty)>1:
        return True
    else:
        return False

# adds a constraint to the matrix
def constrain(table,eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1
        # set up counter to iterate through the total length of rows
        j = 0
        while j < lr:
            # Iterate by row
            row_check = table[j,:]
            # total will be sum of entries in row
            total = 0
            # Find first row with all 0 entries
            for i in row_check:
                total += float(i**2)
            if total == 0:
                # We've found the first row with all zero entries
                row = row_check
                break
            j +=1

        eq = convert(eq)
        i = 0
        # iterate through all terms in the constraint function, excluding the last
        while i<len(eq)-1:
            # assign row values according to the equation
            row[i] = eq[i]
            i +=1
        #row[len(eq)-1] = 1
        row[-1] = eq[-1]

        # add slack variable according to location in tableau.
        row[var+j] = 1
    else:
        print('Cannot add another constraint.')

# checks to determine if an objective function can be added to the matrix
def add_obj(table):
    lr = len(table[:,0])
    # want to know IF exactly one row of all zero elements exist
    empty = []
    # iterate through each row
    for i in range(lr):
        total = 0
        for j in table[i,:]:
            # use squared value so (-x) and (+x) don't cancel each other out
            total += j**2
        if total == 0:
            # append zero to list ONLY if all elements in a row are zero
            empty.append(total)
    # There is exactly one row with all zero elements if the following is true
    if len(empty)==1:
        return True
    else:
        return False

# adds the onjective functio nto the matrix.
def obj(table,eq):
    if add_obj(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0
    # iterate through all terms in the constraint function, excluding the last
        while i<len(eq)-1:
            # assign row values according to the equation
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding constraints before the objective function can be added.')

# solves maximization problem for optimal solution, returns dictionary w/ keys x1,x2...xn and max.
def maxz(table, output='summary'):
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)

    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1,-1]
    for k,v in val.items():
        val[k] = round(v,6)
    if output == 'table':
        return table
    else:
        return val

# solves minimization problems for optimal solution, returns dictionary w/ keys x1,x2...xn and min.
def minz(table, output='summary'):
    table = convert_min(table)

    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)

    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['min'] = table[-1,-1]*-1
    for k,v in val.items():
        val[k] = round(v,6)
    if output == 'table':
        return table
    else:
        return val

if __name__ == "__main__":

    m = gen_matrix(2,2)
    constrain(m,'2,-1,G,10')
    constrain(m,'1,1,L,20')
    obj(m,'5,10,0')
    print(maxz(m))

    m = gen_matrix(2,4)
    constrain(m,'2,5,G,30')
    constrain(m,'-3,5,G,5')
    constrain(m,'8,3,L,85')
    constrain(m,'-9,7,L,42')
    obj(m,'2,7,0')
    print(minz(m))

{'x1': 10.0, 'x2': 10.0, 'max': 150.0}
{'x1': 5.0, 'x2': 4.0, 'min': 38.0}


In [ ]:
"""Simplex method for  linear programming
The *simplex* method uses a traditional, full-tableau implementation of
Dantzig's simplex algorithm [1]_, [2]_ (*not* the Nelder-Mead simplex).
This algorithm is included for backwards compatibility and educational
purposes.
    .. versionadded:: 0.15.0
Warnings
--------
The simplex method may encounter numerical difficulties when pivot
values are close to the specified tolerance. If encountered try
remove any redundant constraints, change the pivot strategy to Bland's
rule or increase the tolerance value.
Alternatively, more robust methods maybe be used. See
:ref:`'interior-point' <optimize.linprog-interior-point>` and
:ref:`'revised simplex' <optimize.linprog-revised_simplex>`.
References
----------
.. [1] Dantzig, George B., Linear programming and extensions. Rand
       Corporation Research Study Princeton Univ. Press, Princeton, NJ,
       1963
.. [2] Hillier, S.H. and Lieberman, G.J. (1995), "Introduction to
       Mathematical Programming", McGraw-Hill, Chapter 4.
"""

import numpy as np
from warnings import warn
from scipy.optimize import OptimizeResult, OptimizeWarning
from scipy.optimize.optimize import _check_unknown_options
from scipy.optimize._linprog_util import _postsolve


def _pivot_col(T, tol=1e-9, bland=False):
    """
    Given a linear programming simplex tableau, determine the column
    of the variable to enter the basis.
    Parameters
    ----------
    T : 2-D array
        A 2-D array representing the simplex tableau, T, corresponding to the
        linear programming problem. It should have the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],    0]]
        for a Phase 2 problem, or the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],   0],
         [c'[0],  c'[1], ...,  c'[n_total],  0]]
         for a Phase 1 problem (a problem in which a basic feasible solution is
         sought prior to maximizing the actual objective. ``T`` is modified in
         place by ``_solve_simplex``.
    tol : float
        Elements in the objective row larger than -tol will not be considered
        for pivoting. Nominally this value is zero, but numerical issues
        cause a tolerance about zero to be necessary.
    bland : bool
        If True, use Bland's rule for selection of the column (select the
        first column with a negative coefficient in the objective row,
        regardless of magnitude).
    Returns
    -------
    status: bool
        True if a suitable pivot column was found, otherwise False.
        A return of False indicates that the linear programming simplex
        algorithm is complete.
    col: int
        The index of the column of the pivot element.
        If status is False, col will be returned as nan.
    """
    ma = np.ma.masked_where(T[-1, :-1] >= -tol, T[-1, :-1], copy=False)
    if ma.count() == 0:
        return False, np.nan
    if bland:
        # ma.mask is sometimes 0d
        return True, np.nonzero(np.logical_not(np.atleast_1d(ma.mask)))[0][0]
    return True, np.ma.nonzero(ma == ma.min())[0][0]


def _pivot_row(T, basis, pivcol, phase, tol=1e-9, bland=False):
    """
    Given a linear programming simplex tableau, determine the row for the
    pivot operation.
    Parameters
    ----------
    T : 2-D array
        A 2-D array representing the simplex tableau, T, corresponding to the
        linear programming problem. It should have the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],    0]]
        for a Phase 2 problem, or the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],   0],
         [c'[0],  c'[1], ...,  c'[n_total],  0]]
         for a Phase 1 problem (a Problem in which a basic feasible solution is
         sought prior to maximizing the actual objective. ``T`` is modified in
         place by ``_solve_simplex``.
    basis : array
        A list of the current basic variables.
    pivcol : int
        The index of the pivot column.
    phase : int
        The phase of the simplex algorithm (1 or 2).
    tol : float
        Elements in the pivot column smaller than tol will not be considered
        for pivoting. Nominally this value is zero, but numerical issues
        cause a tolerance about zero to be necessary.
    bland : bool
        If True, use Bland's rule for selection of the row (if more than one
        row can be used, choose the one with the lowest variable index).
    Returns
    -------
    status: bool
        True if a suitable pivot row was found, otherwise False. A return
        of False indicates that the linear programming problem is unbounded.
    row: int
        The index of the row of the pivot element. If status is False, row
        will be returned as nan.
    """
    if phase == 1:
        k = 2
    else:
        k = 1
    ma = np.ma.masked_where(T[:-k, pivcol] <= tol, T[:-k, pivcol], copy=False)
    if ma.count() == 0:
        return False, np.nan
    mb = np.ma.masked_where(T[:-k, pivcol] <= tol, T[:-k, -1], copy=False)
    q = mb / ma
    min_rows = np.ma.nonzero(q == q.min())[0]
    if bland:
        return True, min_rows[np.argmin(np.take(basis, min_rows))]
    return True, min_rows[0]


def _apply_pivot(T, basis, pivrow, pivcol, tol=1e-9):
    """
    Pivot the simplex tableau inplace on the element given by (pivrow, pivol).
    The entering variable corresponds to the column given by pivcol forcing
    the variable basis[pivrow] to leave the basis.
    Parameters
    ----------
    T : 2-D array
        A 2-D array representing the simplex tableau, T, corresponding to the
        linear programming problem. It should have the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],    0]]
        for a Phase 2 problem, or the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],   0],
         [c'[0],  c'[1], ...,  c'[n_total],  0]]
         for a Phase 1 problem (a problem in which a basic feasible solution is
         sought prior to maximizing the actual objective. ``T`` is modified in
         place by ``_solve_simplex``.
    basis : 1-D array
        An array of the indices of the basic variables, such that basis[i]
        contains the column corresponding to the basic variable for row i.
        Basis is modified in place by _apply_pivot.
    pivrow : int
        Row index of the pivot.
    pivcol : int
        Column index of the pivot.
    """
    basis[pivrow] = pivcol
    pivval = T[pivrow, pivcol]
    T[pivrow] = T[pivrow] / pivval
    for irow in range(T.shape[0]):
        if irow != pivrow:
            T[irow] = T[irow] - T[pivrow] * T[irow, pivcol]

    # The selected pivot should never lead to a pivot value less than the tol.
    if np.isclose(pivval, tol, atol=0, rtol=1e4):
        message = (
            "The pivot operation produces a pivot value of:{0: .1e}, "
            "which is only slightly greater than the specified "
            "tolerance{1: .1e}. This may lead to issues regarding the "
            "numerical stability of the simplex method. "
            "Removing redundant constraints, changing the pivot strategy "
            "via Bland's rule or increasing the tolerance may "
            "help reduce the issue.".format(pivval, tol))
        warn(message, OptimizeWarning, stacklevel=5)


def _solve_simplex(T, n, basis, callback, postsolve_args,
                   maxiter=1000, tol=1e-9, phase=2, bland=False, nit0=0,
                   ):
    """
    Solve a linear programming problem in "standard form" using the Simplex
    Method. Linear Programming is intended to solve the following problem form:
    Minimize::
        c @ x
    Subject to::
        A @ x == b
            x >= 0
    Parameters
    ----------
    T : 2-D array
        A 2-D array representing the simplex tableau, T, corresponding to the
        linear programming problem. It should have the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],    0]]
        for a Phase 2 problem, or the form:
        [[A[0, 0], A[0, 1], ..., A[0, n_total], b[0]],
         [A[1, 0], A[1, 1], ..., A[1, n_total], b[1]],
         .
         .
         .
         [A[m, 0], A[m, 1], ..., A[m, n_total], b[m]],
         [c[0],   c[1], ...,   c[n_total],   0],
         [c'[0],  c'[1], ...,  c'[n_total],  0]]
         for a Phase 1 problem (a problem in which a basic feasible solution is
         sought prior to maximizing the actual objective. ``T`` is modified in
         place by ``_solve_simplex``.
    n : int
        The number of true variables in the problem.
    basis : 1-D array
        An array of the indices of the basic variables, such that basis[i]
        contains the column corresponding to the basic variable for row i.
        Basis is modified in place by _solve_simplex
    callback : callable, optional
        If a callback function is provided, it will be called within each
        iteration of the algorithm. The callback must accept a
        `scipy.optimize.OptimizeResult` consisting of the following fields:
            x : 1-D array
                Current solution vector
            fun : float
                Current value of the objective function
            success : bool
                True only when a phase has completed successfully. This
                will be False for most iterations.
            slack : 1-D array
                The values of the slack variables. Each slack variable
                corresponds to an inequality constraint. If the slack is zero,
                the corresponding constraint is active.
            con : 1-D array
                The (nominally zero) residuals of the equality constraints,
                that is, ``b - A_eq @ x``
            phase : int
                The phase of the optimization being executed. In phase 1 a basic
                feasible solution is sought and the T has an additional row
                representing an alternate objective function.
            status : int
                An integer representing the exit status of the optimization::
                     0 : Optimization terminated successfully
                     1 : Iteration limit reached
                     2 : Problem appears to be infeasible
                     3 : Problem appears to be unbounded
                     4 : Serious numerical difficulties encountered
            nit : int
                The number of iterations performed.
            message : str
                A string descriptor of the exit status of the optimization.
    postsolve_args : tuple
        Data needed by _postsolve to convert the solution to the standard-form
        problem into the solution to the original problem.
    maxiter : int
        The maximum number of iterations to perform before aborting the
        optimization.
    tol : float
        The tolerance which determines when a solution is "close enough" to
        zero in Phase 1 to be considered a basic feasible solution or close
        enough to positive to serve as an optimal solution.
    phase : int
        The phase of the optimization being executed. In phase 1 a basic
        feasible solution is sought and the T has an additional row
        representing an alternate objective function.
    bland : bool
        If True, choose pivots using Bland's rule [3]_. In problems which
        fail to converge due to cycling, using Bland's rule can provide
        convergence at the expense of a less optimal path about the simplex.
    nit0 : int
        The initial iteration number used to keep an accurate iteration total
        in a two-phase problem.
    Returns
    -------
    nit : int
        The number of iterations. Used to keep an accurate iteration total
        in the two-phase problem.
    status : int
        An integer representing the exit status of the optimization::
         0 : Optimization terminated successfully
         1 : Iteration limit reached
         2 : Problem appears to be infeasible
         3 : Problem appears to be unbounded
         4 : Serious numerical difficulties encountered
    """
    nit = nit0
    status = 0
    message = ''
    complete = False

    if phase == 1:
        m = T.shape[1]-2
    elif phase == 2:
        m = T.shape[1]-1
    else:
        raise ValueError("Argument 'phase' to _solve_simplex must be 1 or 2")

    if phase == 2:
        # Check if any artificial variables are still in the basis.
        # If yes, check if any coefficients from this row and a column
        # corresponding to one of the non-artificial variable is non-zero.
        # If found, pivot at this term. If not, start phase 2.
        # Do this for all artificial variables in the basis.
        # Ref: "An Introduction to Linear Programming and Game Theory"
        # by Paul R. Thie, Gerard E. Keough, 3rd Ed,
        # Chapter 3.7 Redundant Systems (pag 102)
        for pivrow in [row for row in range(basis.size)
                       if basis[row] > T.shape[1] - 2]:
            non_zero_row = [col for col in range(T.shape[1] - 1)
                            if abs(T[pivrow, col]) > tol]
            if len(non_zero_row) > 0:
                pivcol = non_zero_row[0]
                _apply_pivot(T, basis, pivrow, pivcol, tol)
                nit += 1

    if len(basis[:m]) == 0:
        solution = np.empty(T.shape[1] - 1, dtype=np.float64)
    else:
        solution = np.empty(max(T.shape[1] - 1, max(basis[:m]) + 1),
                            dtype=np.float64)

    while not complete:
        # Find the pivot column
        pivcol_found, pivcol = _pivot_col(T, tol, bland)
        if not pivcol_found:
            pivcol = np.nan
            pivrow = np.nan
            status = 0
            complete = True
        else:
            # Find the pivot row
            pivrow_found, pivrow = _pivot_row(T, basis, pivcol, phase, tol, bland)
            if not pivrow_found:
                status = 3
                complete = True

        if callback is not None:
            solution[:] = 0
            solution[basis[:n]] = T[:n, -1]
            x = solution[:m]
            x, fun, slack, con = _postsolve(
                x, postsolve_args
            )
            res = OptimizeResult({
                'x': x,
                'fun': fun,
                'slack': slack,
                'con': con,
                'status': status,
                'message': message,
                'nit': nit,
                'success': status == 0 and complete,
                'phase': phase,
                'complete': complete,
                })
            callback(res)

        if not complete:
            if nit >= maxiter:
                # Iteration limit exceeded
                status = 1
                complete = True
            else:
                _apply_pivot(T, basis, pivrow, pivcol, tol)
                nit += 1
    return nit, status


def _linprog_simplex(c, c0, A, b, callback, postsolve_args,
                     maxiter=1000, tol=1e-9, disp=False, bland=False,
                     **unknown_options):
    """
    Minimize a linear objective function subject to linear equality and
    non-negativity constraints using the two phase simplex method.
    Linear programming is intended to solve problems of the following form:
    Minimize::
        c @ x
    Subject to::
        A @ x == b
            x >= 0
    User-facing documentation is in _linprog_doc.py.
    Parameters
    ----------
    c : 1-D array
        Coefficients of the linear objective function to be minimized.
    c0 : float
        Constant term in objective function due to fixed (and eliminated)
        variables. (Purely for display.)
    A : 2-D array
        2-D array such that ``A @ x``, gives the values of the equality
        constraints at ``x``.
    b : 1-D array
        1-D array of values representing the right hand side of each equality
        constraint (row) in ``A``.
    callback : callable, optional
        If a callback function is provided, it will be called within each
        iteration of the algorithm. The callback function must accept a single
        `scipy.optimize.OptimizeResult` consisting of the following fields:
            x : 1-D array
                Current solution vector
            fun : float
                Current value of the objective function
            success : bool
                True when an algorithm has completed successfully.
            slack : 1-D array
                The values of the slack variables. Each slack variable
                corresponds to an inequality constraint. If the slack is zero,
                the corresponding constraint is active.
            con : 1-D array
                The (nominally zero) residuals of the equality constraints,
                that is, ``b - A_eq @ x``
            phase : int
                The phase of the algorithm being executed.
            status : int
                An integer representing the status of the optimization::
                     0 : Algorithm proceeding nominally
                     1 : Iteration limit reached
                     2 : Problem appears to be infeasible
                     3 : Problem appears to be unbounded
                     4 : Serious numerical difficulties encountered
            nit : int
                The number of iterations performed.
            message : str
                A string descriptor of the exit status of the optimization.
    postsolve_args : tuple
        Data needed by _postsolve to convert the solution to the standard-form
        problem into the solution to the original problem.
    Options
    -------
    maxiter : int
       The maximum number of iterations to perform.
    disp : bool
        If True, print exit status message to sys.stdout
    tol : float
        The tolerance which determines when a solution is "close enough" to
        zero in Phase 1 to be considered a basic feasible solution or close
        enough to positive to serve as an optimal solution.
    bland : bool
        If True, use Bland's anti-cycling rule [3]_ to choose pivots to
        prevent cycling. If False, choose pivots which should lead to a
        converged solution more quickly. The latter method is subject to
        cycling (non-convergence) in rare instances.
    unknown_options : dict
        Optional arguments not used by this particular solver. If
        `unknown_options` is non-empty a warning is issued listing all
        unused options.
    Returns
    -------
    x : 1-D array
        Solution vector.
    status : int
        An integer representing the exit status of the optimization::
         0 : Optimization terminated successfully
         1 : Iteration limit reached
         2 : Problem appears to be infeasible
         3 : Problem appears to be unbounded
         4 : Serious numerical difficulties encountered
    message : str
        A string descriptor of the exit status of the optimization.
    iteration : int
        The number of iterations taken to solve the problem.
    References
    ----------
    .. [1] Dantzig, George B., Linear programming and extensions. Rand
           Corporation Research Study Princeton Univ. Press, Princeton, NJ,
           1963
    .. [2] Hillier, S.H. and Lieberman, G.J. (1995), "Introduction to
           Mathematical Programming", McGraw-Hill, Chapter 4.
    .. [3] Bland, Robert G. New finite pivoting rules for the simplex method.
           Mathematics of Operations Research (2), 1977: pp. 103-107.
    Notes
    -----
    The expected problem formulation differs between the top level ``linprog``
    module and the method specific solvers. The method specific solvers expect a
    problem in standard form:
    Minimize::
        c @ x
    Subject to::
        A @ x == b
            x >= 0
    Whereas the top level ``linprog`` module expects a problem of form:
    Minimize::
        c @ x
    Subject to::
        A_ub @ x <= b_ub
        A_eq @ x == b_eq
         lb <= x <= ub
    where ``lb = 0`` and ``ub = None`` unless set in ``bounds``.
    The original problem contains equality, upper-bound and variable constraints
    whereas the method specific solver requires equality constraints and
    variable non-negativity.
    ``linprog`` module converts the original problem to standard form by
    converting the simple bounds to upper bound constraints, introducing
    non-negative slack variables for inequality constraints, and expressing
    unbounded variables as the difference between two non-negative variables.
    """
    _check_unknown_options(unknown_options)

    status = 0
    messages = {0: "Optimization terminated successfully.",
                1: "Iteration limit reached.",
                2: "Optimization failed. Unable to find a feasible"
                   " starting point.",
                3: "Optimization failed. The problem appears to be unbounded.",
                4: "Optimization failed. Singular matrix encountered."}

    n, m = A.shape

    # All constraints must have b >= 0.
    is_negative_constraint = np.less(b, 0)
    A[is_negative_constraint] *= -1
    b[is_negative_constraint] *= -1

    # As all constraints are equality constraints the artificial variables
    # will also be basic variables.
    av = np.arange(n) + m
    basis = av.copy()

    # Format the phase one tableau by adding artificial variables and stacking
    # the constraints, the objective row and pseudo-objective row.
    row_constraints = np.hstack((A, np.eye(n), b[:, np.newaxis]))
    row_objective = np.hstack((c, np.zeros(n), c0))
    row_pseudo_objective = -row_constraints.sum(axis=0)
    row_pseudo_objective[av] = 0
    T = np.vstack((row_constraints, row_objective, row_pseudo_objective))

    nit1, status = _solve_simplex(T, n, basis, callback=callback,
                                  postsolve_args=postsolve_args,
                                  maxiter=maxiter, tol=tol, phase=1,
                                  bland=bland
                                  )
    # if pseudo objective is zero, remove the last row from the tableau and
    # proceed to phase 2
    nit2 = nit1
    if abs(T[-1, -1]) < tol:
        # Remove the pseudo-objective row from the tableau
        T = T[:-1, :]
        # Remove the artificial variable columns from the tableau
        T = np.delete(T, av, 1)
    else:
        # Failure to find a feasible starting point
        status = 2
        messages[status] = (
            "Phase 1 of the simplex method failed to find a feasible "
            "solution. The pseudo-objective function evaluates to {0:.1e} "
            "which exceeds the required tolerance of {1} for a solution to be "
            "considered 'close enough' to zero to be a basic solution. "
            "Consider increasing the tolerance to be greater than {0:.1e}. "
            "If this tolerance is unacceptably  large the problem may be "
            "infeasible.".format(abs(T[-1, -1]), tol)
        )

    if status == 0:
        # Phase 2
        nit2, status = _solve_simplex(T, n, basis, callback=callback,
                                      postsolve_args=postsolve_args,
                                      maxiter=maxiter, tol=tol, phase=2,
                                      bland=bland, nit0=nit1
                                      )

    solution = np.zeros(n + m)
    solution[basis[:n]] = T[:n, -1]
    x = solution[:m]

    return x, status, messages[status], int(nit2)

In [ ]:
from scipy.optimize import linprog
import numpy as np
from math import trunc
ans = {'global': -9999999999, 'x': [4]}
def branch(c, A, b, x, mod):
  global ans
  res = linprog( c, A, b, bounds = x, method = 'simplex')
  print(res)
  if not res.success:
    return
  else:
    flag = 0
    for i in range(len(res.x)):
      if flag == 1:
        break
      if abs(round(res.x[i]) - res.x[i]) > 10 ** -6:
        flag = 1
        x1 = x[:]
        x1[i] = [0, trunc(res.x[i])]
        branch(c, A, b, x1, mod)
        x2 = x[:]
        x2[i] = [trunc(res.x[i]) + 1, None]
        branch(c, A, b, x2, mod)
    if flag == 0 and abs(round(res.fun) - res.fun) < 10 ** -6:
      print('asdasdasdsad', abs(round(res.fun) - res.fun))
      if mod == 'min':
        if round(res.fun) > ans['global']:
          ans['global'] = round(res.fun)
          ans['x'] = res.x
      else:
        if round(res.fun) < ans['global']:
          ans['global'] = round(res.fun)
          ans['x'] = res.x
        
      


mod = 'max'
    
c = np.array([0, 2, 0, 0])



if mod == 'max':
  c *= -1
  ans['global'] *= -1
A = [[3, 2, 1, 0], [3, 2, 0, 1]]
b = [6, 0]
x = [[0, None], [0, None],[0,None], [0,None]]
res = linprog( c, A, b, bounds = x, method = 'simplex')

print(res)

     con: array([], dtype=float64)
     fun: 0.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([0., 0.])
  status: 0
 success: True
       x: array([0., 0., 6., 0.])


In [ ]:
def get_matrix(av, Jb):
  a = np.eye(len(Jb))
  j = 0
  for i in Jb:
    a[:, j] = av[:, i - 1]
    j += 1
  a = np.linalg.inv(a)
  return a

In [ ]:
def main_simplex(A, c, x, jb):
  iteration = 0
  while True:
    iteration += 1
    if iteration == 1:
      ab = np.array([np.copy(A[:,i-1]) for i in jb]).transpose()
      A_i = np.linalg.inv(ab)
    else:
      ab[:,position_min_tetta] = A[:, jb[position_min_tetta] - 1]
      A_i = solve(A_i, ab[:,position_min_tetta], position_min_tetta + 1)
    cb = np.array([c[i - 1] for i in jb])
    u = np.dot(cb, A_i)
    delta = np.dot(u, A) -c
    j0=0
    if min(delta) >= 0:
      
      return (x, jb)
    while delta[j0]>=0:
      j0 += 1
    z = np.dot(A_i, A[:,j0])
    tetta = [float(x[jb[i] - 1])/float(z[i]) if z[i]> 0 else np.inf for i in range(len(jb))]
    tetta0 = np.min(tetta)
    if tetta0 is np.inf:
      print("целевая функция неограничена на множестве допустимых планов")
    position_min_tetta = np.where(tetta == tetta0)[0][0]
    jb_new = np.copy(jb)
    jb_new[position_min_tetta] = j0 + 1
    x_new = np.copy(x)
    j = 0
    x_new = x_new.astype(np.float64)
    for i in jb:
      x_new[i - 1] = float(x[i - 1]) - tetta0*float(z[j])
      j+=1
    x_new[j0] = tetta0
    x = x_new
    jb = jb_new

In [ ]:
import numpy as np

def begin_simplex(A, b):
  n = len(A[0])
  m = len(b)
  for i in range(m):
    if b[i] < 0:
      b[i] *= -1
      A[i] *= -1
  E = np.eye(m)
  Av = np.empty([m, n + m])
  for i in range(m):
    Av[i] = np.append(A[i], E[i])
  
  x = np.append(np.zeros(n), b)
  
  Jb = np.array([n + i + 1 for i in range(m)])
  c = [0 if i < n else -1 for i in range(n + m)]
  x, jb = main_simplex(Av, c, x, Jb)
  
  for i in x[n:]:
    if abs(i) > 10 ** -6:
      return 'Несовместна'
  while True:
    k = -1
    for i in range(len(jb)):
      if (jb[i] > n):
        k = i
    if k == -1:
      return x[:n], jb
      break
    
    jnb = []
    for i in range(1, n + 1):
      if not i in jb:
        jnb.append(i)
   
    avb_inv = get_matrix(Av, jb)
    flag = False
    for i in jnb:
      l = np.dot(avb_inv, Av[:,i - 1])
      
      if abs(l[k]) < 10 ** -6:
        jb[k] = i
        flag = True
        break
    
    if not flag:
      c = c[:-1]
      jb = jb[:-1]
      A = A[:-1]
      Av = Av[:-1]
      b = b[:-1]

In [ ]:
c = np.array([1, 2, 0, 0])
A = np.array([[3, 1, 0, 0], [1, 3, 0, 0]])
b = np.array([7, 7])

A_copy = np.copy(A)
B_copy = np.copy(b)
e = begin_simplex(A, b)
A = A_copy
b = B_copy

x = e[0]
Jb = e[1]
print(x, Jb)
Jb = sorted(Jb)
print(x, Jb)

[1.75 1.75 0.   0.   0.   0.  ] [1 2]
[1.75 1.75 0.   0.  ] [1 2]
[1.75 1.75 0.   0.  ] [1, 2]


In [ ]:
#c = np.array([21, 11,0])
#A = np.array([[7, 4, 1]])
#b = np.array([13])
#c *= -1
#bound = [[0, None], [0, None], [0, None]]
res = linprog( c, A, b, bounds = bound, method = 'simplex')
print(res)
x = res.x
Jb = []
for i in range(len(x)):
  if x[i] != 0:
    Jb.append(i + 1)


     con: array([], dtype=float64)
     fun: -33.00000000000001
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([-1.77635684e-15, -1.11022302e-16,  0.00000000e+00])
  status: 0
 success: True
       x: array([0., 3., 1., 1., 0.])


In [ ]:
ab = np.array([np.copy(A[:,i-1]) for i in Jb]).transpose()
print(ab)

[[ 4.          1.          0.        ]
 [-0.57142857 -0.14285714  1.        ]
 [ 0.         -0.25       -0.25      ]]


In [ ]:
ab_inv = np.linalg.inv(ab)
print(ab_inv)

[[ 2.85714286e-01  2.50000000e-01  1.00000000e+00]
 [-1.42857143e-01 -1.00000000e+00 -4.00000000e+00]
 [ 1.42857143e-01  1.00000000e+00  2.22044605e-16]]


In [ ]:
k = None
fl = 0
for i in range(len(x)):
  if abs(round(x[i]) - x[i]) > 10 ** -6:
    k = i
    fl = 1
    
    break

if fl == 0:
  
  print(x)


[0. 3. 1. 1. 0.]


In [ ]:
l = [0 for i in range(len(Jb))]
l[k] = 1
print(l)

[0, 1]


In [ ]:
y = np.dot(l, ab_inv)
print(y)

[ 0.   -1.75]


In [ ]:
small_a = []
for i in range(len(A[0])):
  small_a.append(np.dot(y, A[:,i]))
print(small_a)


[0.0, 1.0, 0.2499999999999999, -1.75]


In [ ]:
small_b = np.dot(y, b)
print(small_b)

1.4999999999999998


In [ ]:
Jnb = [i for i in range(1, len(c) + 1) if i not in Jb]
print(Jnb)

[3, 4]


In [ ]:
new_a = [0 for _ in range(len(A[0]))]
for i in range(len(Jnb)):
  new_a[Jnb[i] - 1] = -get_f(small_a[Jnb[i] - 1])
new_a += [1]
print(new_a)


[0, 0, -0.2499999999999999, -0.25, 1]


In [ ]:
arr = A.tolist()
for i in arr:
  i += [0]
arr.append(new_a)
A = np.array(arr)
print(A)

[[ 7.          4.          1.          0.          0.        ]
 [ 0.         -0.57142857 -0.14285714  1.          0.        ]
 [ 0.          0.         -0.25       -0.25        1.        ]]


In [ ]:
arr = c.tolist()
arr += [0]
c = np.array(arr)
print(arr)

[-21, -11, 0, 0, 0]


In [ ]:
arr = b.tolist()
arr += [-get_f(small_b)]
b = np.array(arr)
bound.append([0, None])
print(b)

[13.         -0.85714286 -0.5       ]


In [ ]:
xk = 'None'
k = 'None'
ram = 546556
for i in range(len(x)):
  if x[i] != round(x[i]):
    xk = i + 1
    ram = x[i]
    break
if xk != 'None':
  for i in range(len(Jb)):
    if Jb[i] == xk:
      k = i
      break
print(k)
  

0


In [ ]:
ab = np.array([np.copy(A[:,i-1]) for i in Jb]).transpose()
print(ab)


[[7]]


In [ ]:
Jnb = [i for i in range(1, len(c) + 1) if i not in Jb]
print(Jnb)

[2, 3]


In [ ]:
anb = np.array([np.copy(A[:,i-1]) for i in Jnb]).transpose()
print(anb)

[[4 1]]


In [ ]:
xb

NameError: ignored

In [ ]:
ab_inv = np.linalg.inv(ab)
print(ab_inv)

[[ 0.375 -0.125]
 [-0.125  0.375]]


In [ ]:
e = np.dot(ab_inv, anb)
print(e)

[[0. 0.]
 [0. 0.]]


In [ ]:
l = e[k]
print(l)

[0. 0.]


In [ ]:
kk = [0 for i in range(len(c))]
for i in range(len(Jnb)):
  kk[Jnb[i] - 1] = get_f(l[i])
  
arr = A.tolist()
arr.append(kk)
for i in range(len(arr) - 1):
  arr[i].append(0)
arr[-1].append(-1)

A = np.array(arr)
arr = c.tolist()
arr.append(0)
c = np.array(arr)
print(A)
  


[[ 3.  1.  0.  0.  0.]
 [ 1.  3.  0.  0.  0.]
 [ 0.  0.  1.  1. -1.]]


In [ ]:
arr = b.tolist()
arr.append(get_f(ram))
b = np.array(arr)
print(b)
bound.append([0, None])

[7.   7.   0.75]


In [ ]:
print(x, Jb)
for i in Jb:
  print(i, round(x[i - 1]) - x[i - 1])
  if abs(round(x[i - 1]) - x[i - 1]) > 10 ** -6:
    iz = i - 1
    break
  else:
    print('Идеальное решение')
print(iz)


[1. 1. 2. 0. 0.] [1, 2, 3]
1 1.1102230246251565e-16
Идеальное решение
2 0.0
Идеальное решение
3 -1.7763568394002505e-15
Идеальное решение


NameError: ignored

Выбираем Jk что не удовлетворяет плану

Матрицу а разобьем на 2 части
базисеые столбцы и не базисные столбцы
так же разбивается вектор x
так же х с крышкой
xн = 0
ab * xб = b
xb = ab(inv) * b
домножаем обе части на обратную базисную

In [ ]:
e = [0 for i in range(len(Jb))]
e[iz] = 1
print(e)

[1]


In [ ]:
print(ab_inv[iz])
y_mask = np.multiply(e, ab_inv[iz])
y = [0 for i in range(len(A[0]))]
for i in range(len(Jb)):
  y[Jb[i] - 1] = y_mask[i]
print(y)

[0.14285714]
[0.14285714285714285, 0, 0]


In [ ]:
print(A[:,iz])
small_a = []
for i in range(len(A[0])):
  small_a.append(y[i] A[:,i]))
print(small_a)
small_b = np.dot(y, b)


SyntaxError: ignored

In [ ]:
ee = []
for i in range(0, len(A[0])):
  if i + 1 not in Jb:
    ee.append(-get_f(small_a[i]))
print(ee)

In [ ]:
arr = A.tolist()
for i in range(len(A)):

  arr[i].append(0)
A = np.array(arr)
new_a = []
for i in range(len(A[0]) - 1):
  if i + 1 in Jb:
    new_a.append(0)
  else:
    new_a.append(ee[0])
    ee.pop(0)
new_a.append(1)
arr = A.tolist()
arr.append(new_a)
A = np.array(arr)
arr = b.tolist()
arr.append(-get_f(small_b[0]))
b = np.array(arr)

print(A, b)
print(A[1][1])